# Прогноз срока экспозиции объявления
Каждый пользователь, размещая объявление об аренде квартиры, хочет понимать, сколько времени потребуется для сдачи объекта. В первую очередь такая информация нужна на форме подачи объявлений, что заметно ограничивает набор возможных признаков. Вам предлагается построить модель, прогнозирующую длительность экспозиции объявлений на Яндекс.Недвижимости.
Для построения целевой переменной срок экспозиции разбит на несколько классов, каждому из которых соответствует целое число: "меньше 7 дней"(1), "7-14 дней"(2), "15-30 дней"(3), "30-70 дней"(4), "более 70 дней"(5).
Метрика, по которой оцениваются решения, записывается следующим образом:

$$metric = \frac{1}l \sum_{i=0}^{l}{exp^{|prediction_i - target_i|}}$$

### Формат ввода
В файле с обучающей выборкой в каждой строке через табуляцию записаны числовой id объявления и набор признаков, характеризующих его.

Данные в тестовой выборке exposition_test.tsv представлены в аналогичном формате, за исключением целевой переменной.

### Формат вывода
Первая строка должна содержать названия столбцов: id target.
В каждой последующей строке файла через табуляцию должны быть записаны id объявления и предсказанный класс в виде вещественного числа.

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn_som.som import SOM
import hdbscan
from sklearn.preprocessing import MinMaxScaler

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на',
          round(start_mem - end_mem, 2),
          'Мб (минус',
          round(100 * (start_mem - end_mem) / start_mem, 1),
         '%)')
    return df

### Загрузка данных

In [3]:
### Используем очищенные данные
# Заполнены пропуски
# Текстовые атрибуты преобразованы в единичные вектора
# Добавлены мета-признаки даты
# Уточнены цены
# Добавллена средняя цена по метро/городу
# Удалены слабо коррелириующие признаки

train_data = pd.read_csv('./data/exposition_train.basic.csv')
test_data = pd.read_csv('./data/exposition_test.csv')

In [4]:
train_data.columns

Index(['total_area', 'ceiling_height', 'rooms', 'living_area', 'price',
       'day_mean', 'doy_108', 'price_locality_name_median', 'target',
       'day_count'],
      dtype='object')

### Выравнивание тестовых данных

In [5]:
test_data['day'] =  pd.to_datetime(test_data['day'])
test_data['doy_108'] = test_data['day'].dt.dayofyear

In [6]:
# Построим прогноз day_mean по day_count
test_data['day_count'] = test_data.groupby('day')['total_area'].transform('count')

In [7]:
x = np.array(train_data['day_count']).reshape(-1, 1)
y = train_data['day_mean']
model = LinearRegression().fit(x, y)
train_data = train_data.drop(labels=['day_count'], axis=1)

In [8]:
test_data['day_mean'] = model.predict(np.array(test_data['day_count']).reshape(-1, 1))
test_data = test_data.drop(labels=['day_count'], axis=1)

In [9]:
cols = train_data.columns.to_list()
cols.remove('target')
print(cols)
test_data = test_data[cols]

['total_area', 'ceiling_height', 'rooms', 'living_area', 'price', 'day_mean', 'doy_108', 'price_locality_name_median']


In [10]:
# Оптимизация памяти
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)

Потребление памяти меньше на 18.7 Мб (минус 76.4 %)
Потребление памяти меньше на 3.21 Мб (минус 73.4 %)


In [11]:
all_data = pd.concat([train_data, test_data])
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428166 entries, 0 to 71665
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   total_area                  428166 non-null  float16
 1   ceiling_height              428166 non-null  float16
 2   rooms                       428166 non-null  int8   
 3   living_area                 428166 non-null  float16
 4   price                       428166 non-null  int32  
 5   day_mean                    428166 non-null  float16
 6   doy_108                     428166 non-null  int16  
 7   price_locality_name_median  428166 non-null  float16
 8   target                      356500 non-null  float64
dtypes: float16(5), float64(1), int16(1), int32(1), int8(1)
memory usage: 13.5 MB


### Нормализация данных

In [12]:
all_data = all_data.drop(labels=['doy_108'], axis=1)
all_data_mm = pd.DataFrame(MinMaxScaler().fit_transform(all_data[all_data.columns[:-1]]))

### Построим кластеры по всем данным

In [13]:
# np.random.seed(42)
# som = SOM(m=10, n=10, dim=len(all_data_mm.columns), max_iter=1000)
# labels = som.fit_predict(np.array(all_data_mm), epochs=100, shuffle=False)

hdb = hdbscan.HDBSCAN(min_cluster_size=5)
hdb.fit_predict(all_data_mm)
labels = hdb.labels_

In [14]:
all_data_mm["label"] = labels

train_data_mm = all_data_mm[:len(train_data)]
train_data_mm["target"] = train_data["target"]

test_data_mm = all_data_mm[:-len(test_data)]

/home/eugeney/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [15]:
train_data_mm.head()

,0,1,2,3,4,5,6,label,target
0,0.097782,0.107143,0.6,0.112360,0.002111,0.454021,0.002111,-1,1
1,0.032258,0.107143,0.2,0.043153,0.000555,0.641781,0.000933,4036,2
2,0.029832,0.094308,0.0,0.042697,0.000577,0.662288,0.000578,996,2
3,0.072581,0.107143,0.6,0.110112,0.000777,0.665492,0.001167,-1,2
4,0.092742,0.107143,0.6,0.110112,0.001777,0.466838,0.001778,2501,3


### Предсказание класса
Выбор наиболее популярного класса в кластере

In [45]:
### Самый популярный класс в кластере
# groups = train_data_mm.groupby(["label","target"]).count()[0]
# clusters_popular = [0]*len(labels)
# clusters_class = [0]*len(labels)
# for group in groups.iteritems():
#     items = group[1]
#     cluster = group[0][0]
#     if items > clusters_popular[cluster]:
#         clusters_popular[cluster] = items
#         clusters_class[cluster] = group[0][1]

In [46]:
# train_data_mm["target_pred"] = train_data_mm["label"].apply(lambda x:clusters_class[x])
# test_data_mm["target"] = test_data_mm["label"].apply(lambda x:clusters_class[x])

In [41]:
### Среднее значение класса по кластеру
clusters = train_data_mm.groupby("label").mean()["target"]
train_data_mm['target_pred'] = train_data_mm['label'].apply(lambda x: clusters.loc[x])
test_data_mm['target'] = test_data_mm['label'].apply(lambda x: clusters.loc[x])

Оценка предсказания на тренировочных данных

In [42]:
print ("Оценка точности: ", np.exp(np.abs(train_data_mm["target_pred"] - train_data_mm["target"])).sum() / len(train_data_mm))
# target_pred = 3     4.28

Оценка точности:  4.053223052605551


In [26]:
test_data_mm['target'] = np.around(test_data_mm['target'])
test_data_mm.head()

/home/eugeney/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,0,1,2,3,4,5,6,label,target
0,0.097782,0.107143,0.6,0.112360,0.002111,0.454021,0.002111,-1,3.0
1,0.032258,0.107143,0.2,0.043153,0.000555,0.641781,0.000933,4036,3.0
2,0.029832,0.094308,0.0,0.042697,0.000577,0.662288,0.000578,996,3.0
3,0.072581,0.107143,0.6,0.110112,0.000777,0.665492,0.001167,-1,3.0
4,0.092742,0.107143,0.6,0.110112,0.001777,0.466838,0.001778,2501,3.0


### Сохранение результатов

In [48]:
test_data_mm['target'].to_csv('./data/exposition_sample_submision.tsv', sep='\t', index_label='id')